In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, udf
from pyspark.sql.types import IntegerType

In [2]:
spark = SparkSession.builder.appName('Streaming DF').getOrCreate()

22/12/08 23:42:45 WARN Utils: Your hostname, hasirama resolves to a loopback address: 127.0.1.1; using 192.168.0.219 instead (on interface enp7s0)
22/12/08 23:42:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/08 23:42:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
word = spark.readStream.text('testdata/')
words = word.withColumn('newdata', explode(split(word.value, ' ')))

def get_length(x):
  
  return len(x)

lengthUDF = udf(lambda x: get_length(x), IntegerType())
words = words.withColumn('length', lengthUDF(words.value))

In [4]:
display(words)

DataFrame[value: string, newdata: string, length: int]

In [5]:
query = words.writeStream.format('console').start()
query.awaitTermination(5)

22/12/08 23:42:47 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-9892930d-8a74-43e0-9a09-d9e5b3f60609. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/12/08 23:42:47 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
-------------------------------------------
Batch: 0
-------------------------------------------
+---------+-------+------+
|    value|newdata|length|
+---------+-------+------+
| an apple|     an|     8|
| an apple|  apple|     8|
|an orange|     an|     9|
|an orange| orange|     9|
|  a mango|      a|     7|
|  a mango|  mango|     7|
|    mango|  mango|     5|
+---------+-------+------+



False